In [ ]:
! gh issue list --repo expectedparrot/edsl --state open --limit 1000 --json number,title,state,createdAt,author,body,labels --jq '(["Number","Title","State","Created","Author","Labels","Body"] | @csv), (.[] | [.number, .title, .state, .createdAt, .author.login, (.labels | map(.name) | join(";")), .body] | @csv)' > edsl_open_issues.csv

In [ ]:
from edsl import ScenarioList 
sl = ScenarioList.from_source("csv", "edsl_open_issues.csv")

from edsl import QuestionFreeText, QuestionLinearScale, Survey
q_assess = QuestionFreeText(question_text = """The open-sourced package EDSL is a tool 
for doing AI-powered social science research. It has an open issue on github. 
The title is {{ title }} 
and the description is {{ body }}. 

Please write about how this issue could affect the user experience, 
focusing on ways it would degrade that experience and how severe it might be.
""", 
question_name = "assess")

q_score = QuestionLinearScale(question_text = """The open-sourced package EDSL is a tool 
for doing AI-powered social science research. It has an open issue on github. 
The impact of the issue: `{{ title }}` on users was determined to be: {{ assess.answer }}. 
Please score this issue to help with prioritization.

About 20% of issues should be 1, 
About 30% of issues should be 2. 
About 50% of issues should be 3.
""",
        question_options = [1,2,3],
        option_labels = {
            1: "Tool is fundamentally unusable; affects security and safety",
            2: "Degrades user ability to do useful work; creates confusion",
            3: "An annoyance that will cause problems but users can mostly work around."},
        question_name = "priority")
survey = Survey([q_assess, q_score])
survey.show_flow()